# Fine-tuning Gemma 3 1B for venvy CLI Translation

**Project**: nlcli-wizard  
**Model**: google/gemma-3-1b-it  
**Technique**: QLoRA with Unsloth (Dynamic 4-bit)  
**Hardware**: Google Colab T4 GPU (Free Tier)  

---

## 📚 What You'll Learn

1. **Why Gemma 3 1B?** - Modern SLM optimized for efficiency
2. **What is Unsloth?** - How it makes training 2x faster with 70% less VRAM
3. **QLoRA Explained** - Low-rank adaptation for efficient fine-tuning
4. **4-bit Quantization** - How to compress models without losing accuracy
5. **Dynamic Quantization** - Unsloth's smart approach to preserving critical weights
6. **GGUF Format** - Converting for CPU inference with llama.cpp

---

## 🎯 Training Objective

Fine-tune Gemma 3 1B to translate natural language → venvy CLI commands:

```
Input:  "list all environments sorted by size"
Output: "venvy ls --sort size"
```

**Target Accuracy**: 80-90% on domain-specific commands

---

# Step 1: Setup and Installation

## 🔧 Install Unsloth and Dependencies

### What is Unsloth?

**Unsloth** is a highly optimized library for fine-tuning LLMs that provides:

- **2x Faster Training**: Custom CUDA kernels optimized for LoRA operations
- **70% Less VRAM**: Efficient memory management and gradient checkpointing
- **Dynamic 4-bit Quantization**: Smart weight selection (don't quantize critical layers)
- **Zero Accuracy Loss**: Maintains full precision where it matters

### How Unsloth Works:

```
Traditional Fine-tuning:
├── Load full model (FP16) → 2.2GB VRAM
├── Compute gradients for ALL parameters
└── Update all 1.1B parameters → SLOW

Unsloth + QLoRA:
├── Load model in 4-bit → 650MB VRAM
├── Add small LoRA adapters (8-16MB)
├── Compute gradients ONLY for adapters → FAST
└── Update <1% of parameters → 2x speed, 70% less VRAM
```

### Dynamic 4-bit Quantization:

Unsloth analyzes your model and **selectively avoids quantizing** critical layers:
- Attention output layers
- Layer norms
- Embedding layers

Result: **10% more VRAM but significantly better accuracy**

In [ ]:
# Install Unsloth with all optimizations
# This will take ~3-5 minutes on first run

%%capture
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers trl peft accelerate bitsandbytes

print("✅ Unsloth and dependencies installed!")

In [ ]:
# Verify GPU is available
import torch

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"✅ GPU Available: {gpu_name}")
    print(f"   Total VRAM: {gpu_memory:.1f} GB")
else:
    print("❌ No GPU detected! This notebook requires a GPU.")
    print("   Go to Runtime → Change runtime type → Select T4 GPU")

---

# Step 2: Clone Repository and Load Dataset

You'll use your GitHub personal access token to clone the private repository.

In [ ]:
# Clone your repository with GitHub token
# Replace YOUR_TOKEN and YOUR_USERNAME with your actual values

import os
from getpass import getpass

# Securely input your GitHub token (won't be displayed)
print("Enter your GitHub Personal Access Token:")
GITHUB_TOKEN = getpass()

# Your GitHub username
GITHUB_USERNAME = "pranavkumar2004"  # Change if different

# Clone repository
!git clone https://{GITHUB_TOKEN}@github.com/{GITHUB_USERNAME}/nlcli-wizard.git

# Change to project directory
os.chdir('/content/nlcli-wizard')

print("\n✅ Repository cloned successfully!")
print(f"   Current directory: {os.getcwd()}")

In [ ]:
# Verify dataset exists and inspect it
import json
from pathlib import Path

dataset_path = Path("data/venvy_training.jsonl")

if not dataset_path.exists():
    print("❌ Dataset not found! Make sure you pushed data/venvy_training.jsonl to GitHub")
else:
    # Load and inspect dataset
    examples = []
    with open(dataset_path, 'r') as f:
        for line in f:
            examples.append(json.loads(line))
    
    print(f"✅ Dataset loaded: {len(examples)} examples")
    print("\n📋 Sample Examples:")
    print("-" * 80)
    
    for i, ex in enumerate(examples[:3]):
        print(f"\nExample {i+1}:")
        print(f"  Instruction: {ex['instruction']}")
        print(f"  Output: {ex['output'].strip()}")
    
    print("-" * 80)

---

# Step 3: Load Gemma 3 1B with Unsloth

## 📖 Understanding Model Loading

### What happens when we load a model?

1. **Download from HuggingFace** (~2.2GB for Gemma 3 1B in FP16)
2. **Load into GPU memory** with quantization
3. **Prepare for training** with LoRA adapters

### Quantization Explained:

**Normal Precision (FP16)**:
```
Weight: 0.123456789 (16 bits) → 2 bytes per parameter
1.1B parameters × 2 bytes = 2.2 GB
```

**4-bit Quantization (NF4)**:
```
Weight: 0.123456789 → Quantized to 4 bits (0-15)
1.1B parameters × 0.5 bytes = 550 MB
```

**NF4 (Normal Float 4-bit)**:
- Special quantization format optimized for neural network weights
- Weights follow normal distribution, so use non-uniform quantization
- More precision for common values, less for outliers

### Dynamic 4-bit:

Unsloth's smart feature:
```python
if layer_is_critical():  # Attention, embeddings, norms
    keep_fp16()  # Don't quantize
else:
    quantize_4bit()  # Safe to compress
```

Result: **~650MB VRAM** (instead of 2.2GB) with minimal accuracy loss

In [ ]:
from unsloth import FastLanguageModel
import torch

# Model configuration
model_name = "unsloth/gemma-3-1b-it"  # Unsloth's optimized version
max_seq_length = 512  # Maximum context length for our task
dtype = None  # Auto-detect (FP16 for T4 GPU)
load_in_4bit = True  # Enable 4-bit quantization

print("🔄 Loading Gemma 3 1B with Unsloth optimizations...")
print(f"   Model: {model_name}")
print(f"   Max sequence length: {max_seq_length}")
print(f"   4-bit quantization: {load_in_4bit}")
print("\n⏳ This will take 2-3 minutes (downloading ~2.2GB)...\n")

# Load model with Unsloth
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    # Dynamic 4-bit: Don't quantize critical layers
    # This uses ~10% more VRAM but improves accuracy by 15-20%
)

print("\n✅ Model loaded successfully!")
print(f"   Model parameters: {model.num_parameters():,}")
print(f"   Memory allocated: {torch.cuda.memory_allocated() / 1e9:.2f} GB")

---

# Step 4: Add LoRA Adapters

## 📖 Understanding LoRA (Low-Rank Adaptation)

### The Problem:
Traditional fine-tuning updates **ALL 1.1 billion parameters**:
- Requires massive memory (store gradients for 1.1B params)
- Very slow (update 1.1B weights)
- Easy to overfit on small datasets

### LoRA Solution:
Instead of modifying original weights, add **small adapter matrices**:

```
Original Weight Matrix W (large):
[1024 × 1024] = 1,048,576 parameters

LoRA Decomposition:
ΔW = A × B
A: [1024 × 8]  = 8,192 parameters
B: [8 × 1024]  = 8,192 parameters
Total: 16,384 parameters (64x smaller!)

Final Output:
y = W·x + α·(A·B)·x
    ↑      ↑
 frozen  trainable
```

### Key Parameters:

1. **r (rank)**: Size of adapter matrices (typically 8-16)
   - Higher r = more capacity but slower
   - Lower r = faster but less expressive
   - We use r=16 (good balance)

2. **lora_alpha**: Scaling factor for LoRA updates
   - Controls how much LoRA affects output
   - Typically 2×r (we use 32)

3. **lora_dropout**: Regularization (prevent overfitting)
   - We use 0 (dataset is diverse enough)

4. **target_modules**: Which layers to adapt
   - `q_proj`, `k_proj`: Query/Key attention projections
   - `v_proj`, `o_proj`: Value/Output projections
   - `gate_proj`, `up_proj`, `down_proj`: MLP layers

### Memory Savings:
```
Without LoRA: 1.1B params × 2 bytes = 2.2 GB
With LoRA: 8M params × 2 bytes = 16 MB

Savings: 99.3% reduction in trainable parameters!
```

In [ ]:
# Add LoRA adapters to the model
# These are small matrices we'll train instead of the full model

model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # LoRA rank (size of adapter matrices)
    target_modules=[
        "q_proj",     # Query projection in attention
        "k_proj",     # Key projection
        "v_proj",     # Value projection
        "o_proj",     # Output projection
        "gate_proj",  # MLP gate
        "up_proj",    # MLP up
        "down_proj",  # MLP down
    ],
    lora_alpha=32,  # LoRA scaling factor (typically 2×r)
    lora_dropout=0,  # No dropout (our dataset is diverse)
    bias="none",     # Don't train bias terms
    use_gradient_checkpointing="unsloth",  # Memory optimization
    random_state=42,  # Reproducibility
    use_rslora=False,  # Standard LoRA (RSLoRA is for very large models)
    loftq_config=None,  # No LoftQ quantization
)

print("✅ LoRA adapters added!")
print("\n📊 Model Statistics:")

# Count trainable vs frozen parameters
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())
trainable_pct = 100 * trainable_params / total_params

print(f"   Total parameters: {total_params:,}")
print(f"   Trainable parameters: {trainable_params:,}")
print(f"   Trainable %: {trainable_pct:.2f}%")
print(f"   Memory allocated: {torch.cuda.memory_allocated() / 1e9:.2f} GB")

print("\n💡 Insight:")
print(f"   We're training only {trainable_pct:.2f}% of parameters!")
print(f"   This is why LoRA is so efficient.")

---

# Step 5: Prepare Dataset for Training

## 📖 Understanding the Training Format

### Alpaca Format:
Our dataset uses the Alpaca instruction format:
```json
{
  "instruction": "Task description",
  "input": "Additional context (empty for us)",
  "output": "Expected response"
}
```

### How it's converted for training:
```
Alpaca Format:
  instruction: "Translate to venvy command: list all environments"
  input: ""
  output: "COMMAND: venvy ls\nCONFIDENCE: 0.95\n..."

↓ Transformed to ↓

Gemma 3 Chat Format:
<start_of_turn>user
Translate to venvy command: list all environments<end_of_turn>
<start_of_turn>model
COMMAND: venvy ls
CONFIDENCE: 0.95
EXPLANATION: Lists all registered virtual environments
<end_of_turn>
```

### Why this format?
- Gemma 3 is trained as a chat model with turn-based conversation
- `<start_of_turn>user` signals user input
- `<start_of_turn>model` signals model response
- This matches how Gemma 3 was pre-trained

In [ ]:
from datasets import load_dataset

# Load dataset from JSONL file
dataset = load_dataset('json', data_files='data/venvy_training.jsonl', split='train')

print(f"✅ Dataset loaded: {len(dataset)} examples")
print("\n📋 Dataset Structure:")
print(dataset)
print("\n📝 Sample Example:")
print(dataset[0])

In [ ]:
# Split dataset: 90% train, 10% validation
# Validation set helps us monitor if the model is overfitting

dataset = dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = dataset['train']
eval_dataset = dataset['test']

print(f"✅ Dataset split:")
print(f"   Training examples: {len(train_dataset)}")
print(f"   Validation examples: {len(eval_dataset)}")

print("\n💡 Why validation set?")
print("   We'll evaluate on this during training to detect overfitting.")
print("   If validation loss stops improving, we stop training.")

In [ ]:
# Format dataset for Gemma 3 chat format
# This converts our Alpaca format to Gemma's expected input format

# Gemma 3 chat template
alpaca_prompt = """<start_of_turn>user
{}<end_of_turn>
<start_of_turn>model
{}<end_of_turn>"""

EOS_TOKEN = tokenizer.eos_token  # End-of-sequence token

def formatting_prompts_func(examples):
    """
    Convert Alpaca format to Gemma 3 chat format.
    
    For each example:
    1. Combine instruction + input (input is empty for us)
    2. Format as Gemma chat turn
    3. Add EOS token for proper training
    """
    instructions = examples["instruction"]
    inputs = examples["input"]
    outputs = examples["output"]
    
    texts = []
    for instruction, input_text, output in zip(instructions, inputs, outputs):
        # Combine instruction and input (input is empty for our dataset)
        full_instruction = instruction + ("\n" + input_text if input_text else "")
        
        # Format as chat turns
        text = alpaca_prompt.format(full_instruction, output) + EOS_TOKEN
        texts.append(text)
    
    return {"text": texts}

# Apply formatting to both train and validation sets
train_dataset = train_dataset.map(
    formatting_prompts_func,
    batched=True,
)

eval_dataset = eval_dataset.map(
    formatting_prompts_func,
    batched=True,
)

print("✅ Dataset formatted for Gemma 3 chat!")
print("\n📝 Formatted Example:")
print("-" * 80)
print(train_dataset[0]['text'])
print("-" * 80)

---

# Step 6: Configure Training Parameters

## 📖 Understanding Hyperparameters

### Key Training Parameters:

#### 1. **Learning Rate (lr)**: How fast the model learns
```
Too high (1e-3):  Model diverges, loss explodes
Just right (2e-4): Smooth learning, converges well
Too low (1e-5):   Learns too slowly, wastes time
```
We use **2e-4** (0.0002) - standard for LoRA fine-tuning.

#### 2. **Batch Size**: How many examples per update
```
per_device_batch_size=4:  Process 4 examples at once
gradient_accumulation_steps=4: Accumulate 4 batches
Effective batch size = 4 × 4 = 16
```
Why split?
- T4 GPU has 16GB VRAM
- Batch size 16 would cause OOM (out of memory)
- So we process 4 at a time, accumulate gradients, then update

#### 3. **Epochs**: How many times to see full dataset
```
1 epoch = model sees each example once
3 epochs = model sees each example 3 times
```
We use **3 epochs** - enough to learn without overfitting.

#### 4. **Weight Decay**: Regularization to prevent overfitting
```
weight_decay=0.01: Small penalty on large weights
```
Encourages model to use many small weights instead of few large ones.

#### 5. **Learning Rate Schedule**: Warmup + Cosine Decay
```
Step 0-50:    Warmup (gradual increase) → Prevents early instability
Step 50-end:  Cosine decay (gradual decrease) → Better convergence

Learning Rate over time:
    |
2e-4|        _______________
    |      /                 \
    |    /                     \
    |  /                         \
  0 |_/____________________________\_____
     0   50                   1000  steps
```

#### 6. **Mixed Precision (FP16)**: Speed + Memory optimization
```
Normal (FP32):  32 bits per number → Slow but accurate
Mixed (FP16):   16 bits per number → 2x faster, 2x less memory
```
T4 GPU has FP16 cores (Tensor Cores) → much faster.

### Expected Training Time:
```
1,350 examples × 3 epochs = 4,050 training steps
4,050 / (batch_size 16) = ~253 update steps
~1-2 seconds per step on T4
Total: ~8-10 minutes
```

In [ ]:
from transformers import TrainingArguments
from trl import SFTTrainer

# Training configuration
training_args = TrainingArguments(
    # Output and logging
    output_dir="./outputs",              # Where to save model checkpoints
    logging_dir="./logs",                # Where to save logs
    logging_steps=10,                    # Log every 10 steps
    
    # Training hyperparameters
    num_train_epochs=3,                  # Train for 3 epochs
    per_device_train_batch_size=4,       # 4 examples per GPU
    gradient_accumulation_steps=4,       # Accumulate 4 batches (effective batch=16)
    learning_rate=2e-4,                  # Standard LoRA learning rate
    weight_decay=0.01,                   # L2 regularization
    
    # Learning rate schedule
    lr_scheduler_type="cosine",          # Cosine decay schedule
    warmup_steps=50,                     # Warmup for first 50 steps
    
    # Optimization
    optim="adamw_8bit",                  # 8-bit AdamW (saves memory)
    fp16=True,                           # Mixed precision training (2x faster)
    
    # Evaluation
    eval_strategy="steps",               # Evaluate during training
    eval_steps=50,                       # Evaluate every 50 steps
    per_device_eval_batch_size=4,        # Batch size for evaluation
    
    # Checkpointing
    save_strategy="steps",               # Save checkpoints
    save_steps=100,                      # Save every 100 steps
    save_total_limit=3,                  # Keep only 3 best checkpoints
    load_best_model_at_end=True,         # Load best checkpoint at end
    metric_for_best_model="eval_loss",   # Use validation loss to pick best
    
    # Memory optimizations
    gradient_checkpointing=True,         # Save memory (slight speed cost)
    max_grad_norm=1.0,                   # Gradient clipping (stability)
    
    # Reproducibility
    seed=42,
    
    # Disable unnecessary features
    report_to="none",                    # Don't report to wandb/tensorboard
)

print("✅ Training configuration set!")
print("\n📊 Training Summary:")
print(f"   Epochs: {training_args.num_train_epochs}")
print(f"   Effective batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"   Learning rate: {training_args.learning_rate}")
print(f"   Warmup steps: {training_args.warmup_steps}")
print(f"   FP16 enabled: {training_args.fp16}")

# Calculate approximate training time
total_steps = (len(train_dataset) * training_args.num_train_epochs) // (training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps)
print(f"\n⏱️ Estimated training time:")
print(f"   Total steps: ~{total_steps}")
print(f"   Time: ~{total_steps * 2 / 60:.1f} minutes (assuming 2 sec/step)")

---

# Step 7: Train the Model! 🚀

## 📖 What Happens During Training?

### Training Loop:
```python
for epoch in range(3):
    for batch in train_dataset:
        # 1. Forward pass: Get model predictions
        predictions = model(batch)
        
        # 2. Calculate loss: How wrong are we?
        loss = cross_entropy(predictions, targets)
        
        # 3. Backward pass: Calculate gradients
        gradients = loss.backward()
        
        # 4. Update LoRA weights
        optimizer.step(gradients)
        
        # 5. Log progress
        if step % 10 == 0:
            print(f"Loss: {loss:.4f}")
```

### What to Watch:

1. **Training Loss**: Should decrease smoothly
   ```
   Good:    2.5 → 1.8 → 1.2 → 0.8 → 0.5
   Bad:     2.5 → 5.8 → NaN (model diverged!)
   ```

2. **Validation Loss**: Should also decrease
   ```
   Good:    Train loss ≈ Val loss (not overfitting)
   Bad:     Train 0.3, Val 2.5 (overfitting!)
   ```

3. **Speed**: Should be ~1-2 seconds per step
   - Slower? GPU not being used efficiently
   - Faster? Might be skipping computation

### Training Metrics Explained:

- **loss**: Cross-entropy loss (lower = better)
- **learning_rate**: Current LR (starts low, increases, then decreases)
- **epoch**: Which epoch we're on (0-3)
- **grad_norm**: Gradient magnitude (should be stable, not exploding)

This cell will take ~8-10 minutes. Grab a coffee! ☕

In [ ]:
# Create trainer with SFTTrainer (Supervised Fine-Tuning)
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    dataset_text_field="text",  # Which field contains the formatted text
    max_seq_length=max_seq_length,
    args=training_args,
    packing=False,  # Don't pack multiple examples (our examples are short)
)

print("✅ Trainer initialized!")
print("\n🚀 Starting training...")
print("   This will take ~8-10 minutes on T4 GPU")
print("   Watch the loss decrease over time!")
print("\n" + "="*80)

In [ ]:
# Start training!
# The output will show:
# - Loss (should decrease)
# - Learning rate (should follow warmup + cosine schedule)
# - Time per step
# - Memory usage

trainer_stats = trainer.train()

print("\n" + "="*80)
print("🎉 Training complete!")
print("\n📊 Final Statistics:")
print(f"   Train runtime: {trainer_stats.metrics['train_runtime']:.2f} seconds")
print(f"   Train samples/second: {trainer_stats.metrics['train_samples_per_second']:.2f}")
print(f"   Final train loss: {trainer_stats.metrics['train_loss']:.4f}")

# Get validation metrics
eval_results = trainer.evaluate()
print(f"\n📈 Validation Results:")
print(f"   Validation loss: {eval_results['eval_loss']:.4f}")
print(f"   Validation perplexity: {eval_results.get('eval_perplexity', 'N/A')}")

---

# Step 8: Test the Model

Let's see if our fine-tuned model can actually translate natural language to venvy commands!

In [ ]:
# Enable inference mode (faster, less memory)
FastLanguageModel.for_inference(model)

def test_command_translation(nl_query):
    """
    Test the model's ability to translate natural language to venvy commands.
    """
    # Format as instruction
    instruction = f"Translate to venvy command: {nl_query}"
    
    # Format as Gemma chat turn
    prompt = alpaca_prompt.format(instruction, "")
    
    # Tokenize
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
    
    # Generate
    outputs = model.generate(
        **inputs,
        max_new_tokens=128,
        temperature=0.1,  # Low temperature for deterministic output
        top_p=0.9,
        do_sample=True,
    )
    
    # Decode
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Extract model response (after "<start_of_turn>model")
    if "<start_of_turn>model" in response:
        response = response.split("<start_of_turn>model")[-1].strip()
    
    return response

print("✅ Inference mode enabled!")
print("\n🧪 Testing model on example queries...\n")
print("="*80)

In [ ]:
# Test on various queries
test_queries = [
    "list all environments",
    "show me venvs sorted by size",
    "register this venv",
    "what environment am i using",
    "clean up old environments",
    "scan home directory for venvs",
    "show statistics",
    "setup shell integration for zsh",
]

for query in test_queries:
    response = test_command_translation(query)
    print(f"Query: {query}")
    print(f"Response:\n{response}")
    print("-"*80)

print("\n💡 Insight:")
print("   Check if the COMMAND: lines are correct venvy commands!")
print("   Target accuracy: 80-90% on these queries")

---

# Step 9: Save the Fine-tuned Model

We'll save both:
1. **LoRA adapters only** (small, ~16MB)
2. **Merged model** (base + adapters, ~2.2GB)

In [ ]:
# Save LoRA adapters only (small file, quick to upload/download)
model.save_pretrained("venvy_gemma3_lora")
tokenizer.save_pretrained("venvy_gemma3_lora")

print("✅ LoRA adapters saved to: venvy_gemma3_lora/")
print("   Size: ~16MB (adapters only)")
print("\n💡 To load later:")
print("   model = FastLanguageModel.from_pretrained('venvy_gemma3_lora')")

In [ ]:
# Merge LoRA adapters into base model (for GGUF conversion)
print("🔄 Merging LoRA adapters into base model...")
print("   This combines the base Gemma 3 1B with our trained adapters")
print("   Result will be ~2.2GB in FP16 format")

model.save_pretrained_merged(
    "venvy_gemma3_merged",
    tokenizer,
    save_method="merged_16bit",  # Save in FP16 (2 bytes per param)
)

print("\n✅ Merged model saved to: venvy_gemma3_merged/")
print("   Size: ~2.2GB (full model in FP16)")
print("\n💡 Next step: Convert to GGUF for CPU inference")

---

# Step 10: Convert to GGUF Format

## 📖 Understanding GGUF Conversion

### Why GGUF?
**GGUF** (GPT-Generated Unified Format) is optimized for CPU inference:
- Used by llama.cpp for efficient CPU/Metal/Vulkan inference
- Supports various quantization levels (2-bit to 8-bit)
- Memory-mapped for fast loading
- Cross-platform (Windows, Mac, Linux)

### Quantization Options:
```
Q2_K: 2-bit → ~300MB, fast but lower quality
Q3_K_M: 3-bit → ~450MB, good balance
Q4_0: 4-bit basic → ~550MB, standard
Q4_K_M: 4-bit with K-means → ~600MB, better quality ✅ (our choice)
Q5_K_M: 5-bit with K-means → ~700MB, excellent quality
Q8_0: 8-bit → ~1.1GB, minimal loss
```

### K-means Quantization:
Instead of uniform quantization, K-means clusters weights:
```
Standard Q4: [-1.0, -0.5, 0.0, 0.5, 1.0] (uniform bins)
K-means Q4:  [-0.9, -0.3, 0.1, 0.6, 1.2] (optimized bins)
                                         ↑
                        Better matches weight distribution
```

### Importance Matrix (imatrix):
Identifies which layers are most important for your specific task:
1. Run inference on your dataset
2. Measure activation magnitudes per layer
3. Quantize unimportant layers more aggressively
4. Preserve critical layers with higher precision

Result: **15-20% better quality** at same size

In [ ]:
# Install llama.cpp for GGUF conversion
%%capture
!git clone https://github.com/ggerganov/llama.cpp
!cd llama.cpp && make

print("✅ llama.cpp installed!")

In [ ]:
# Step 1: Convert HuggingFace model to GGUF FP16
print("🔄 Step 1: Converting to GGUF FP16 format...")

!python llama.cpp/convert_hf_to_gguf.py \
    venvy_gemma3_merged \
    --outfile venvy_gemma3_fp16.gguf \
    --outtype f16

print("\n✅ GGUF FP16 model created: venvy_gemma3_fp16.gguf (~2.2GB)")

In [ ]:
# Step 2: Generate importance matrix from our dataset
print("🔄 Step 2: Generating importance matrix...")
print("   This analyzes which layers are critical for venvy commands")
print("   Takes ~5-10 minutes...\n")

# Create a text file with sample commands for imatrix generation
import json

with open('imatrix_data.txt', 'w') as f:
    # Use 100 random examples from our dataset
    for i, example in enumerate(train_dataset.select(range(min(100, len(train_dataset))))):
        f.write(example['text'] + '\n\n')

print("✅ Created imatrix_data.txt with 100 examples")

# Generate importance matrix
!cd llama.cpp && ./llama-imatrix \
    -m ../venvy_gemma3_fp16.gguf \
    -f ../imatrix_data.txt \
    -o ../venvy_imatrix.dat \
    --chunks 100

print("\n✅ Importance matrix generated: venvy_imatrix.dat")
print("   This will help preserve quality in critical layers")

In [ ]:
# Step 3: Quantize to Q4_K_M with importance matrix
print("🔄 Step 3: Quantizing to Q4_K_M with importance matrix...")
print("   This will create the final ~600MB model for CPU inference\n")

!cd llama.cpp && ./llama-quantize \
    ../venvy_gemma3_fp16.gguf \
    ../venvy_gemma3_q4km.gguf \
    Q4_K_M \
    --imatrix ../venvy_imatrix.dat

print("\n✅ Quantized model created: venvy_gemma3_q4km.gguf")

# Check file sizes
import os
fp16_size = os.path.getsize('venvy_gemma3_fp16.gguf') / 1e9
q4km_size = os.path.getsize('venvy_gemma3_q4km.gguf') / 1e9
compression_ratio = fp16_size / q4km_size

print(f"\n📊 Compression Statistics:")
print(f"   Original (FP16): {fp16_size:.2f} GB")
print(f"   Quantized (Q4_K_M): {q4km_size:.2f} GB")
print(f"   Compression ratio: {compression_ratio:.1f}x")
print(f"\n💡 Quality:")
print(f"   Q4_K_M with imatrix preserves ~95% of FP16 quality")
print(f"   Perfect for CPU inference with minimal accuracy loss")

---

# Step 11: Test GGUF Model

Let's verify the quantized model works correctly!

In [ ]:
# Install llama-cpp-python for testing
%%capture
!pip install llama-cpp-python

print("✅ llama-cpp-python installed!")

In [ ]:
from llama_cpp import Llama

# Load GGUF model
print("🔄 Loading GGUF model...")

llm = Llama(
    model_path="venvy_gemma3_q4km.gguf",
    n_ctx=512,  # Context window
    n_threads=4,  # CPU threads
    verbose=False,
)

print("✅ GGUF model loaded!")
print(f"   Model size: {q4km_size:.2f} GB")
print(f"   Context window: 512 tokens")

In [ ]:
# Test GGUF model on queries
def test_gguf_translation(nl_query):
    instruction = f"Translate to venvy command: {nl_query}"
    prompt = alpaca_prompt.format(instruction, "")
    
    response = llm(
        prompt,
        max_tokens=128,
        temperature=0.1,
        top_p=0.9,
        stop=["<end_of_turn>", "\n\n"],
    )
    
    return response['choices'][0]['text'].strip()

print("🧪 Testing GGUF model...\n")
print("="*80)

test_queries = [
    "list all environments",
    "register this venv as myproject",
    "show current environment",
]

for query in test_queries:
    response = test_gguf_translation(query)
    print(f"Query: {query}")
    print(f"Response: {response}")
    print("-"*80)

print("\n💡 If the responses look correct, your GGUF model is ready!")

---

# Step 12: Download the Models

Download these files to your local machine:
1. `venvy_gemma3_q4km.gguf` - Final quantized model (~600MB)
2. `venvy_gemma3_lora/` - LoRA adapters (~16MB)

You can also push them to your GitHub repository.

In [ ]:
# Option 1: Download via Colab UI
from google.colab import files

print("📥 Downloading models...")
print("   This may take a few minutes for the 600MB GGUF file\n")

# Download GGUF model
files.download('venvy_gemma3_q4km.gguf')

print("\n✅ Model downloaded!")
print("\n💡 To push to GitHub:")
print("   1. Create models/ directory in your repo")
print("   2. Add venvy_gemma3_q4km.gguf to models/")
print("   3. Use Git LFS for large files (>100MB)")
print("   4. Or host on HuggingFace Model Hub")

In [ ]:
# Option 2: Push to HuggingFace Hub (recommended for large models)
# Requires HuggingFace account and token

from huggingface_hub import HfApi, create_repo

# Uncomment and run if you want to upload to HuggingFace
# HF_TOKEN = getpass("Enter your HuggingFace token: ")
# HF_USERNAME = "your-username"
# REPO_NAME = "venvy-gemma3-q4km"

# api = HfApi()
# create_repo(f"{HF_USERNAME}/{REPO_NAME}", token=HF_TOKEN, private=True)
# api.upload_file(
#     path_or_fileobj="venvy_gemma3_q4km.gguf",
#     path_in_repo="venvy_gemma3_q4km.gguf",
#     repo_id=f"{HF_USERNAME}/{REPO_NAME}",
#     token=HF_TOKEN,
# )

print("💡 Hosting on HuggingFace is recommended for:")
print("   - Easy sharing and versioning")
print("   - Direct download via HF API")
print("   - Model cards and documentation")

---

# 🎉 Training Complete!

## What You Accomplished:

1. ✅ **Fine-tuned Gemma 3 1B** on 1,350 venvy command examples
2. ✅ **Used QLoRA** for efficient training (99.3% parameter reduction)
3. ✅ **Leveraged Unsloth** for 2x speed, 70% less VRAM
4. ✅ **Quantized to Q4_K_M** with importance matrix
5. ✅ **Created GGUF model** for CPU inference (~600MB)

## What You Learned:

### 1. Unsloth Benefits:
- Custom CUDA kernels for 2x faster LoRA operations
- Dynamic 4-bit quantization (smart layer selection)
- Gradient checkpointing for 70% VRAM reduction

### 2. QLoRA Mechanics:
- Low-rank decomposition (A × B instead of full W)
- Train only 0.7% of parameters (8M vs 1.1B)
- NF4 quantization for base model (4-bit compressed)

### 3. Quantization Techniques:
- **4-bit Quantization**: 4x compression with minimal loss
- **K-means Clustering**: Optimized bins for weight distribution
- **Importance Matrix**: Preserve critical layers
- **GGUF Format**: Optimized for CPU inference

### 4. Training Best Practices:
- Learning rate warmup prevents early instability
- Cosine decay improves final convergence
- Gradient accumulation enables larger effective batch size
- Mixed precision (FP16) doubles speed on modern GPUs

## Next Steps:

1. **Integrate with venvy** - Add NL parser using llama-cpp-python
2. **Test accuracy** - Evaluate on held-out examples
3. **Optimize inference** - Add caching, daemon process
4. **Create demo** - Video showing natural language CLI

## Files to Keep:

```
venvy_gemma3_q4km.gguf        # Final model (~600MB) ✅ IMPORTANT
venvy_gemma3_lora/            # LoRA adapters (~16MB)
venvy_imatrix.dat             # Importance matrix
training_logs.txt             # Training metrics
```

---

**Congratulations! You've successfully fine-tuned a state-of-the-art SLM! 🎊**

Your tech stack is now:
- ✅ 2025 cutting-edge (Gemma 3, Unsloth, Q4_K_M)
- ✅ Production-ready (quantized for CPU inference)
- ✅ Portfolio-worthy (demonstrates advanced ML skills)

**Ready to impress recruiters!** 🚀